In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn import set_config
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
set_config(display='diagram') # Để trực quan hóa pipeline

## Xử lí dữ liệu ban đầu

In [2]:
data_df = pd.read_csv("./Data/Laptops.csv")
data_df.head()

,URL,Name,Cost,Thương hiệu,Bảo hành,Màu sắc,Series laptop,Part-number,Thế hệ CPU,CPU,...,Kích thước,Pin,Khối lượng,Bảo mật,Đèn LED trên máy,Phụ kiện đi kèm,Tính năng,Mic,Ổ đĩa quang,Mô tả bảo hành
0,https://phongvu.vn//may-tinh-xach-tay-laptop-a...,"Laptop ASUS VivoBook X407MA-BV043T (14"" HD/N40...",6.390.000đ,ASUS,24,Vàng đồng,VivoBook,BV043T,"Celeron , Intel Celeron N",Intel Celeron N4000 ( 1.1 GHz - 2.6 GHz / 4MB ...,...,32.8 x 24.6 x 2.19 cm,"3 cell 33 Wh , Pin liền",1.5 kg,Vân tay,không đèn,NaN,NaN,NaN,NaN,NaN
1,https://phongvu.vn//may-tinh-xach-tay-laptop-a...,"Laptop ASUS Vivobook E210MA GJ083T ( 11.6"" HD/...",5.990.000đ,ASUS,24,Xanh,VivoBook,GJ083T,"Celeron , Intel Celeron N",Intel Celeron N4020 ( 1.10 GHz - 2.80 GHz / 4M...,...,27.91 x 19.12 x 1.69 cm,"2 cell 38 Wh , Pin liền",1 kg,NaN,không đèn,"Adapter, cáp sạc, tài liệu",Phím số ảo tích hợp trên touchpadMàn hình gập ...,NaN,NaN,NaN
2,https://phongvu.vn//may-tinh-xach-tay-laptop-a...,Laptop ACER Aspire 3 A315-34-C38Y NX.HE3SV.00G...,6.990.000đ,ACER,12,Đen,Aspire 3,NX.HE3SV.00G,"Celeron , Intel Celeron N",Intel Celeron N4020 ( 1.10 GHz - 2.80 GHz / 4M...,...,36.34 x 24.75 x 1.99 cm,"2 cell 37 Wh , Pin liền",1.7 kg,NaN,không đèn,Cáp + Sạc,NaN,NaN,NaN,NaN
3,https://phongvu.vn//may-tinh-xach-tay-laptop-a...,"Laptop ASUS Vivobook X509MA BR270T ( 15.6"" HD/...",6.990.000đ,ASUS,24,Bạc,VivoBook,BR270T,"Celeron , Intel Celeron N",Intel Celeron N4020 ( 1.10 GHz - 2.80 GHz / 4M...,...,36.02 x 23.49 x 2.29 cm,"2 cell 32 Wh , Pin liền",1.6 kg,Vân tay,không đèn,NaN,NaN,NaN,NaN,NaN
4,https://phongvu.vn//may-tinh-xach-tay-laptop-a...,Laptop Acer Aspire 3 A315-34-P3LC (NX.HE3SV.00...,8.090.000đ,ACER,12,Đen,Aspire 3,NX.HE3SV.004,"Pentium , Intel Pentium Silver",Intel Pentium Silver N5000 ( 1.1 GHz - 2.7 GHz...,...,36.34 x 24.75 x 1.99 cm,2 cell 37 Wh,1.7 kg,NaN,không đèn,NaN,NaN,Có,NaN,NaN


In [3]:
data_df.dtypes

URL                           object
Name                          object
Cost                          object
Thương hiệu                   object
Bảo hành                      object
Màu sắc                       object
Series laptop                 object
Part-number                   object
Thế hệ CPU                  object
CPU                           object
Chip đồ họa                 object
RAM                           object
Màn hình                    object
Lưu trữ                      object
Số cổng lưu trữ tối đa    object
Kiểu khe M.2 hỗ trợ           object
Cổng xuất hình             object
Cổng kết nối                  object
Kết nối không dây           object
Bàn phím                    object
Hệ điều hành               object
Kích thước                  object
Pin                           object
Khối lượng                  object
Bảo mật                       object
Đèn LED trên máy              object
Phụ kiện đi kèm               object
T

In [4]:
data_df.duplicated().sum()

0

Vì dữ liệu cột Cost có nhiều giá trị thuộc tính nên ta tiến hành dán nhãn theo từng phân khúc giá để thuận tiện đánh giá mô hình cũng như đảm bảo độ chính xác

In [5]:
# xử lí cột Cost thành dạng numeric
data_df.Cost = data_df.Cost.str.replace(".", "")
data_df.Cost=data_df.Cost.str[:-1]
data_df.Cost =pd.to_numeric(data_df.Cost)
data_df.Cost.value_counts()
# dán nhãn cho cột Cost
def set_label_col(df, col):
    step=3
    labels = range(0, 14, 1)
    start = 4
    end = start
    
    for each in labels:
        end = start+step
        df.loc[(start*(10**6) < df[col])  & (df[col] <= end*(10**6)), "Cost"] = each
        start = end
        
    
    df.loc[df[col] > end*(10**6), col] = 15
set_label_col(data_df, "Cost")
data_df.Cost.value_counts()

3     131
4     114
2      97
5      75
6      73
7      46
8      28
10     24
1      24
15     21
9      18
11     10
12      9
0       7
13      5
Name: Cost, dtype: int64

In [6]:
# Xóa đi dòng cost
y_sr = data_df["Cost"]
X_df = data_df.drop("Cost", axis=1)
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.2, 
                                                              stratify=y_sr, random_state=0)
X_df.shape

(682, 30)

In [7]:
pd.set_option('display.max_colwidth', 200) # Để nhìn rõ hơn
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def num_values(df):
    return df.nunique()
def value_ratios(c):
    return dict((c.value_counts(normalize=True) * 100).round(1))
info = X_df.agg([missing_ratio, num_values, value_ratios])
info

,URL,Name,Thương hiệu,Bảo hành,Màu sắc,Series laptop,Part-number,Thế hệ CPU,CPU,Chip đồ họa,...,Kích thước,Pin,Khối lượng,Bảo mật,Đèn LED trên máy,Phụ kiện đi kèm,Tính năng,Mic,Ổ đĩa quang,Mô tả bảo hành
missing_ratio,0,0,0,0,0.9,7.8,1,1.3,0,1.6,...,0,0,0,49.1,0.1,54.8,93.7,91.6,94.6,99.4
num_values,682,682,9,6,17,59,668,35,82,93,...,379,103,23,3,3,83,31,1,1,4
value_ratios,"{'https://phongvu.vn//may-tinh-xach-tay-laptop-asus-x505za-ej505t-amd-ryzen-5-2500u-xam-xanh-s19030242.html': 0.1, 'https://phongvu.vn//may-tinh-xach-tay-laptop-hp-probook-430-g5-2xr79pa-bac-s1703...","{'Laptop Lenovo ThinkBook 14-IML (20RV00B4VN) (14"" FHD/i5-10210U/4GB/256GB SSD/Intel UHD/Free DOS/1.5kg)': 0.1, 'Laptop Dell Inspiron 3593-70197457 (15"" FHD/i5-1035G1/4GB/1TB HDD/MX230/Win10/2.2kg...","{'ASUS': 25.4, 'HP': 21.8, 'ACER': 17.7, 'Dell': 13.2, 'Lenovo': 10.3, 'MSI': 5.3, 'APPLE': 5.0, 'LG': 1.2, 'Thông số kỹ thuậtThương hiệuHPBảo hành12Thông tin chungMàu sắcBạcSeries laptopProBookPa...","{'12': 68.6, '24': 29.3, '24 tháng': 0.9, '36': 0.6, '12 tháng': 0.4, '36 tháng': 0.1}","{'Bạc': 32.1, 'Đen': 30.6, 'Xám': 18.6, 'Vàng': 10.2, 'Xanh': 3.0, 'Vàng đồng': 1.3, 'Hồng': 0.9, 'Trắng': 0.9, 'Xanh dương': 0.4, 'Xám bạc': 0.4, 'Cam': 0.3, 'Xám đen': 0.3, 'Tím': 0.3, 'Thông s...","{'VivoBook': 14.6, 'ProBook': 7.0, 'Inspiron': 6.2, 'Pavilion': 5.7, 'Vostro': 5.7, 'Ideapad': 5.2, 'ZenBook': 4.6, 'Aspire 5': 4.3, 'Swift 3': 3.7, 'ROG': 3.3, 'Nitro 5': 3.2, 'MacBook Pro': 2.9,...","{'4.72E+12': 0.7, '3493-N4I5122W': 0.3, '242VN': 0.3, 'BQ026T': 0.3, '7591-N5I5591W': 0.1, 'NX.HMHSV.002': 0.1, 'V5I3308W': 0.1, 'EY002T': 0.1, '81N800AAVN': 0.1, 'NX.H2BSV.0002': 0.1, 'CXGR01': 0...","{'Core i5 , Intel Core thế hệ thứ 8': 21.0, 'Core i5 , Intel Core thế hệ thứ 10': 17.5, 'Core i7 , Intel Core thế hệ thứ 10': 8.0, 'Core i3 , Intel Core thế hệ thứ 8': 8.0, 'Core i3 , Intel Core t...","{'Intel Core i5-8265U ( 1.6 GHz - 3.9 GHz / 6MB / 4 nhân, 8 luồng )': 10.1, 'Intel Core i5-8250U ( 1.6 GHz - 3.4 GHz / 6MB / 4 nhân, 8 luồng )': 7.9, 'Intel Core i5-1035G1 ( 1.0 GHz - 3.6 GHz / ...","{'Intel UHD Graphics 620': 20.0, 'Intel UHD Graphics': 19.8, 'Intel Iris Xe Graphics': 4.2, 'Intel HD Graphics 620': 3.4, 'AMD Radeon Graphics': 3.1, 'Intel Iris Plus Graphics': 2.5, 'NVIDIA GeFor...",...,"{'36.16 x 24.56 x 1.79 cm': 1.6, '32.68 x 22.55 x 1.79 cm': 1.5, '36.02 x 23.49 x 2.29 cm': 1.5, '22.59 x 32.4 x 1.99 cm': 1.5, '35.9 x 25.4 x 2.17 cm': 1.3, '36.1 x 24.3 x 1.8 cm': 1.3, '36 x 25....","{'3 cell 42 Wh , Pin liền': 15.0, '3 cell 45 Wh , Pin liền': 8.2, '3 cell 41 Wh , Pin liền': 7.6, '3 cell 48 Wh , Pin liền': 6.2, '2 cell 32 Wh , Pin liền': 3.8, '3 cell 52 Wh , Pin liền': 3...","{'1.7 kg': 13.3, '1.4 kg': 10.6, '1.6 kg': 9.5, '1.5 kg': 8.9, '1.8 kg': 8.1, '2.3 kg': 7.6, '1.9 kg': 7.2, '2.2 kg': 6.3, '1.3 kg': 5.9, '1.2 kg': 5.0, '2.1 kg': 4.0, '2 kg': 3.8, '2.4 kg': 2.8, ...","{'Vân tay': 92.8, 'Khuôn mặt': 6.1, 'Vân tay, Khuôn mặt': 1.2}","{'không đèn': 92.2, 'LED': 5.7, 'RGB': 2.1}","{'Adapter, dây nguồn': 42.2, 'Cáp + sạc': 7.1, 'Cáp + Sạc': 6.5, 'Adapter': 3.9, '30W USB-C Power AdapterUSB-C Charge Cable (2 m)': 2.9, 'Sạc, sách HDSD': 1.9, 'Adapter, dây nguồn, RJ45 Ethernet':...","{'Bàn phím số tích hợp trên touchpad': 14.0, 'Thanh cảm ứng Touch Bar tích hợp Touch ID': 7.0, 'Adapter': 4.7, 'Numberpad tích hợp trên touchpad': 4.7, 'Numberpad tích hợp touchpad': 4.7, 'Numberp...",{'Có': 100.0},{'DVD/CD RW': 100.0},"{'Bh12': 25.0, 'bh12': 25.0, 'BH24T': 25.0, '1': 25.0}"


Ta xử lí dữ liệu để tìm ra những cột có nhiều giá  trị NaN. Ở đây nếu missing ratio > 10 thì ra sẽ xóa cột đó. Ngoài ra ta xóa thêm các cột "Bảo hành, Pin, Khối lượng" vì nó không ảnh hưởng nhiều tới kết quả muốn tìm

In [8]:
drop_cols = ["Bảo hành", "Pin", "Khối lượng"]

for col in X_df.columns:
    if info.loc['missing_ratio'][col] > 10:
        drop_cols.append(col)
drop_cols

['Bảo hành',
 'Pin',
 'Khối lượng',
 'Bảo mật',
 'Phụ kiện đi kèm',
 'Tính năng',
 'Mic',
 'Ổ đĩa quang',
 'Mô tả bảo hành']

# Xử lí tập training

Trong class ColAdderDropper, ta sẽ tiến hành xóa đi các cột có lượng dữ liệu bị thiếu lớn, xóa đi cột "name" và "URL" vì không ảnh hưởng tới bài toán. Ở đây, ta chọn cột "Series laptop" làm cột dữ liệu chính để chọn ra "num top title", đồng thời đổi tên cột này thành "Title"

In [9]:
class ColAdderDropper(BaseEstimator, TransformerMixin):
    def __init__(self, num_top_titles=1, col_titles = 'Series laptop', drop_columns = drop_cols):
        self.num_top_titles = num_top_titles
        self.col_titles = str(col_titles)
        self.drop_columns = drop_cols
    def fit(self, X_df, y=None):
        title_col = X_df[self.col_titles]
        self.title_counts_ = title_col.value_counts()
        titles = list(self.title_counts_.index)
        self.top_titles_ = titles[:max(1, min(self.num_top_titles, len(titles)))]
        return self
    def transform(self, X_df, y=None):
        new_df = X_df.copy()
        new_df.drop(["Name", "URL"], axis=1, inplace=True)
        new_df["Title"] = new_df[self.col_titles]
        new_df.drop([self.col_titles], axis=1, inplace=True)
        
        try:
            new_df = new_df.drop(self.drop_columns, axis = 1)
            self.drop_columns.clear()
        except:
            pass
        
        for each in self.title_counts_.keys():
            if each not in self.top_titles_:
                new_df.loc[new_df['Title'] == each, 'Title'] = "Others"     
        return new_df

In [10]:
col_title = "Series laptop"
num_top_titles = 20
columns = list(set(X_df.columns) - set(drop_cols+["Name","Series laptop", "URL"])) + ["Title"]
# columns = ['Thương hiệu', 'Màu sắc', 'Part-number', 'Thế hệ CPU', 'CPU', 'Chip đồ họa', 'RAM', 'Màn hình', 'Lưu trữ', 
#            'Số cổng lưu trữ tối đa', 'Kiểu khe M.2 hỗ trợ', 'Cổng xuất hình', 
#            'Cổng kết nối', 'Kết nối không dây', 'Bàn phím', 'Hệ điều hành', 'Kích thước', 'Pin', 'Khối lượng', 
#            'Đèn LED trên máy']
print((columns))
steps = [('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
      ('encoder', OneHotEncoder(handle_unknown="ignore"))]
tsf = Pipeline(steps)
transformer = [('tsf', tsf, columns)]
transfer = ColumnTransformer(transformer)
steps = [('col_adderdropper', ColAdderDropper(num_top_titles=num_top_titles, col_titles = col_title)),
        ('transfer', transfer)]
preprocess_pipeline = Pipeline(steps)
preprocess_pipeline
preprocessed_train_X = preprocess_pipeline.fit_transform(train_X_df)
preprocessed_train_X

['Số cổng lưu trữ tối đa', 'CPU', 'Màn hình', 'Chip đồ họa', 'Kích thước', 'Part-number', 'Thế hệ CPU', 'Cổng kết nối', 'Bàn phím', 'Kết nối không dây', 'Thương hiệu', 'Hệ điều hành', 'Lưu trữ', 'Màu sắc', 'Kiểu khe M.2 hỗ trợ', 'RAM', 'Khối lượng', 'Cổng xuất hình', 'Đèn LED trên máy', 'Title']


<545x1678 sparse matrix of type '<class 'numpy.float64'>'
	with 10900 stored elements in Compressed Sparse Row format>

# Xử lí tập validation và mô hình tốt nhất neural netwok

In [11]:
preprocessed_val_X = preprocess_pipeline.fit_transform(val_X_df)
mlpregressor = MLPRegressor(hidden_layer_sizes=(100), solver = 'lbfgs', 
        learning_rate = 'adaptive', random_state=0, max_iter=500, early_stopping = True, verbose = 1)
steps = [('col_adderdropper', ColAdderDropper(num_top_titles=num_top_titles, col_titles = col_title)),
        ('transfer', transfer), ('classifer', mlpregressor)]
full_pipeline = Pipeline(steps)
train_errs = []
val_errs = []

num_top_titles_s = [1, 10, 20, 30, 40, 50]
best_val_err = float('inf'); best_alpha = None; best_num_top_titles = None

for num_top_titles in num_top_titles_s:
        full_pipeline.set_params(col_adderdropper__num_top_titles=num_top_titles)
        full_pipeline.fit(train_X_df, train_y_sr)
        train_err = (1 - full_pipeline.score(train_X_df, train_y_sr))*100
        val_err = (1 - full_pipeline.score(val_X_df, val_y_sr))*100
        if val_err < best_val_err:
            best_val_err = val_err
            best_num_top_titles = num_top_titles
        train_errs.append(train_err)
        val_errs.append(val_err)

In [12]:
full_pipeline.set_params(col_adderdropper__num_top_titles=best_num_top_titles)
full_pipeline.fit(X_df, y_sr)
print(best_num_top_titles)
print(best_val_err)
# best_num_top_titles

10
13.356681693577999


# TESTING

In [13]:
test_df = pd.read_csv("./Data/test.csv")
test_y_sr = test_df["Cost"]
test_X_df = test_df.drop("Cost", axis = 1)

In [14]:
pred_y = full_pipeline.predict(test_X_df)
pd.DataFrame(pred_y ).to_csv("./Data/Predict.csv")
test_err = (1 - full_pipeline.score(test_X_df, test_y_sr))*100
test_err

14.853138975290204

Ta thấy độ lỗi ở mức "có thể chấp nhận" được. Ta nó thể kiểm tra kết quả ở file "Predict.csv"